In [1]:
# Importing google drive to colab
from google.colab import drive
drive.mount('/content/gdrive')
# to change the current working directory for the notebook environment
%cd '/content/gdrive/MyDrive/Colab Notebooks/SciFair/NLP'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/SciFair/NLP


In [2]:
!pip install numpy
!pip install nltk
!pip install tensorflow

!pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing
!pip install Pillow==9.5.0

  Cloning https://github.com/MihaMarkic/tflearn.git (to revision fix/is_sequence_missing) to /tmp/pip-req-build-7wveu5w7
  Running command git clone --filter=blob:none --quiet https://github.com/MihaMarkic/tflearn.git /tmp/pip-req-build-7wveu5w7
  Running command git checkout -b fix/is_sequence_missing --track origin/fix/is_sequence_missing
  Switched to a new branch 'fix/is_sequence_missing'
  Branch 'fix/is_sequence_missing' set up to track remote branch 'fix/is_sequence_missing' from 'origin'.
  Resolved https://github.com/MihaMarkic/tflearn.git to commit 6472b8588e758ff4a33a2764d4ee638bbd0e42f0
  Preparing metadata (setup.py) ... done


In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import numpy
# import tflearn #only works in local
import tensorflow
import random
import json
with open('intents_ZH-TW.json') as file:
    data=json.load(file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
'''
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
'''
words=[]
labels=[]
docs_x=[]
docs_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append (intent["tag"])


In [5]:
words=[stemmer.stem(w.lower()) for w in words if w !="?"]
words=sorted(list(set(words)))
labels=sorted(labels)

In [6]:
training=[]
output=[]
out_empty=[0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1

    training.append(bag)
    output.append(output_row)

In [7]:
training=numpy.array(training)
output=numpy.array(output)

In [8]:
# !pip uninstall tflearn
# !pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing

In [9]:
# !pip install Pillow==9.5.0

In [10]:
import tensorflow as tf
from tensorflow.keras import backend as K
import tflearn

K.clear_session()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model.fit(training,output,n_epoch=1500,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 17999  | total loss: 0.00002 | time: 0.124s
| Adam | epoch: 1500 | loss: 0.00002 - acc: 1.0000 -- iter: 88/94
Training Step: 18000  | total loss: 0.00002 | time: 0.137s
| Adam | epoch: 1500 | loss: 0.00002 - acc: 1.0000 -- iter: 94/94
--


In [12]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [26]:
!apt-get update && apt-get install -y libasound2-dev

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,326 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,161 kB]
Ign:13 https://r2u.st

In [27]:
import time
import speech_recognition as sr
def listenTo():
    r = sr.Recognizer()

    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)

    return r.recognize_google(audio, language='zh-TW')
#Course 88
import tempfile
from gtts import gTTS
from pygame import mixer
mixer.init()

error: ALSA: Couldn't open audio device: No such file or directory

In [30]:
def speak(sentence):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts = gTTS(text=sentence, lang='zh-tw')
        tts.save("{}.mp3".format(fp.name))
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play()

In [34]:
def chat():
    print("開始與您的機器人對話（輸入 結束 以停止）：")
    # speak("開始與您的機器人對話（輸入 結束 以停止）：")
    choice = input("輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：")
    time.sleep(5)
    while True:
        try:
            if choice == '1':  # Text input
                inp = input("請輸入您的問題：")
                if inp=="結束":
                  break
            elif choice == '2':  # Speech input
              inp=listenTo()
              print("You:",inp)
              time.sleep(2)
              if inp=="結束":
                  break
              print("You (語音):", inp)
            else:
              print("無效的選擇，請輸入 '1' 或 '2'。")
              continue  # Skip to the next iteration


            results=model.predict([bag_of_words(inp,words)])
            results_index=numpy.argmax(results)
            tag=labels[results_index]

            for tg in data["intents"]:
                if tg["tag"]==tag:
                    responses=tg["responses"]
            rd=random.choice(responses)
            #speak(rd)
            print(rd)
            time.sleep(5)
        except sr.UnknownValueError:
            print("我無法理解音訊")
        except sr.RequestError as e:
            print("無法從 Google 語音識別服務請求結果; {0}".format(e))
chat()

開始與您的機器人對話（輸入 結束 以停止）：
輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：1
請輸入您的問題：你好
嗨，有什麼我可以幫助你的？
輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：1
請輸入您的問題：我心情不好
即使是簡單的社交互動，也可能對改善心情有幫助。你可以考慮參加一些你感興趣的活動，或者尋找志同道合的人。
輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：1
請輸入您的問題：有什麼解決辦法嗎
我理解有時候人們可能感到社交壓力，但孤獨也可能加劇憂鬱感。有沒有想過嘗試一些小型的社交活動，逐漸擴大你的舒適區？
輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：1
請輸入您的問題：結束


In [31]:
def chat():
    print("開始與您的機器人對話（輸入 結束 以停止）：")
    speak("開始與您的機器人對話（輸入 結束 以停止）：")
    time.sleep(5)
    while True:
        try:
            inp=listenTo()
            print("You:",inp)
            time.sleep(2)
            if inp=="結束":
                break
            results=model.predict([bag_of_words(inp,words)])
            results_index=numpy.argmax(results)
            tag=labels[results_index]

            for tg in data["intents"]:
                if tg["tag"]==tag:
                    responses=tg["responses"]
            rd=random.choice(responses)
            speak(rd)
            print(rd)
            time.sleep(5)
        except sr.UnknownValueError:
            print("我無法理解音訊")
        except sr.RequestError as e:
            print("無法從 Google 語音識別服務請求結果; {0}".format(e))
chat()

開始與您的機器人對話（輸入 結束 以停止）：


error: mixer not initialized

In [22]:
!pip install whisper
!pip install sounddevice wavio
!pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-xkvci1q9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-xkvci1q9
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=fe9dd7a51fe2b4d6f0048f8722b0142899c2c8ec0a5a5b6a25c43049da7a7693
  Stored in directory: /tmp/pip-ephem-wheel-cache-j09ydv09/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [23]:
import random
import time
import numpy as np
import whisper
import sounddevice as sd
import wavio

# Load Whisper model (you can choose 'base', 'small', 'medium', 'large')
whisper_model = whisper.load_model("base")

# Function to record audio from the microphone
def record_audio(duration=5):
    print(f"Recording for {duration} seconds...")
    sample_rate = 44100  # Sample rate in Hz
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # Wait until recording is finished
    filename = "recorded_audio.wav"
    wavio.write(filename, audio, sample_rate, sampwidth=3)  # Save as WAV file
    return filename

# Function to transcribe the recorded audio using Whisper
def transcribe_audio(filename):
    result = whisper_model.transcribe(filename)
    return result['text']

def chat():
    print("開始與您的機器人對話（輸入 結束 以停止）：")
    speak("開始與您的機器人對話（輸入 結束 以停止）：")
    time.sleep(5)

    while True:
        try:
            choice = input("輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：")
            if choice == '1':  # Text input
                inp = input("請輸入您的問題：")
            elif choice == '2':  # Speech input
                filename = record_audio(duration=5)
                inp = transcribe_audio(filename)  # Transcribe the recorded audio
                print("You (語音):", inp)
            else:
                print("無效的選擇，請輸入 '1' 或 '2'。")
                continue  # Skip to the next iteration

            time.sleep(2)
            if inp == "結束":
                break

            # Process the input for the chatbot
            results = model.predict([bag_of_words(inp, words)])
            results_index = np.argmax(results)
            tag = labels[results_index]

            for tg in data["intents"]:
                if tg["tag"] == tag:
                    responses = tg["responses"]

            rd = random.choice(responses)
            speak(rd)
            print(rd)
            time.sleep(5)

        except Exception as e:
            print("發生錯誤:", e)

# Ensure to define your speak function and any other necessary parts of your chatbot


OSError: PortAudio library not found

In [ ]:
chat()